<a href="https://colab.research.google.com/github/yan170506/Sistema-mercadinho../blob/main/Mercadinho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import sqlite3
from datetime import datetime

# --- Funções Auxiliares ---
def input_com_cancelar(prompt, tipo=str, default=None):
    """
    Função para input com opção de cancelamento global.
    Retorna:
        - Valor digitado (convertido para o tipo especificado)
        - None se o usuário digitar '/cancelar'
    """
    while True:
        user_input = input(prompt)
        if user_input.strip().lower() == '/cancelar':
            return None
        try:
            if not user_input and default is not None:
                return default
            return tipo(user_input) if user_input else default
        except ValueError:
            print(f"Erro: Digite um valor válido de {tipo.__name__} ou '/cancelar' para cancelar.")

# --- Operações do Banco de Dados ---
def adicionar_produto():
    print("\n--- Adicionar Novo Produto (digite '/cancelar' a qualquer momento para cancelar a operação) ---")

    nome = input_com_cancelar("Nome do produto: ")
    if nome is None:
        print("❌ Operação cancelada.")
        return

    categoria = input_com_cancelar("Categoria: ")
    if categoria is None:
        print("❌ Operação cancelada.")
        return

    preco = input_com_cancelar("Preço: R$ ", float)
    if preco is None:
        print("❌ Operação cancelada.")
        return

    quantidade = input_com_cancelar("Quantidade em estoque: ", int)
    if quantidade is None:
        print("❌ Operação cancelada.")
        return

    cursor.execute('''
    INSERT INTO produtos (nome, categoria, preco, quantidade, data_cadastro)
    VALUES (?, ?, ?, ?, ?)
    ''', (nome, categoria, preco, quantidade, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

    conn.commit()
    print(f"✅ Produto '{nome}' adicionado com sucesso!")

# ... (outras funções como listar, atualizar, remover)
# Conectar ao banco de dados (será criado se não existir)
conn = sqlite3.connect('mercadinho.db')
cursor = conn.cursor()

# Criar tabela de produtos
cursor.execute('''
CREATE TABLE IF NOT EXISTS produtos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    categoria TEXT,
    preco REAL NOT NULL,
    quantidade INTEGER NOT NULL,
    data_cadastro TEXT
)
''')

# Criar tabela de logs (para a trigger)
cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    acao TEXT NOT NULL,
    produto_id INTEGER,
    produto_nome TEXT,
    quantidade INTEGER,
    data_hora TEXT,
    FOREIGN KEY (produto_id) REFERENCES produtos (id)
)
''')

# Criar trigger para registrar inserções
cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_insercao_produto
AFTER INSERT ON produtos
BEGIN
    INSERT INTO logs (acao, produto_id, produto_nome, quantidade, data_hora)
    VALUES ('INSERT', NEW.id, NEW.nome, NEW.quantidade, datetime('now'));
END;
''')

conn.commit()
print("Banco de dados e tabelas criados com sucesso!")

def menu():
    print("\n--- Sistema de Estoque Mercadinho ---")
    print("1. Adicionar novo produto")
    print("2. Listar todos os produtos")
    print("3. Atualizar produto")
    print("4. Remover produto")
    print("5. Visualizar logs")
    print("6. Sair")
    return input("Escolha uma opção: ")

def adicionar_produto():
    print("\n--- Adicionar Novo Produto (digite '/cancelar' a qualquer momento para cancelar a operação) ---")

    nome = input_com_cancelar("Nome do produto: ")
    if nome is None:
        print("❌ Operação cancelada.")
        return

    categoria = input_com_cancelar("Categoria: ")
    if categoria is None:
        print("❌ Operação cancelada.")
        return

    preco = input_com_cancelar("Preço: R$ ", float)
    if preco is None:
        print("❌ Operação cancelada.")
        return

    quantidade = input_com_cancelar("Quantidade em estoque: ", int)
    if quantidade is None:
        print("❌ Operação cancelada.")
        return

    cursor.execute('''
    INSERT INTO produtos (nome, categoria, preco, quantidade, data_cadastro)
    VALUES (?, ?, ?, ?, ?)
    ''', (nome, categoria, preco, quantidade, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

    conn.commit()
    print(f"✅ Produto '{nome}' adicionado com sucesso!")


def listar_produtos():
    print("/n--- Lista de produtos ---")
    cursor.execute('SELECT * FROM produtos')
    produtos = cursor.fetchall()

    if not produtos:
      print ("Nenhum produto cadastrado")
    else:
      for produto in produtos:
        print(f"ID: {produto [1]}) | Nome: {produto [1]} | Categoria: {produto [2]} | "
              f"Preço R$ {produto [3]:.2f} | Quantidade: {produto [4]} | "
              f"Cadastrado em: {produto[5]}")
def atualizar_produto():
    print("\n--- Atualizar Produto (digite '/cancelar' a qualquer momento) ---")

    # Lista produtos (já com tratamento de cancelamento)
    listar_produtos()

    produto_id = input_com_cancelar("\nID do produto a atualizar: ", int)
    if produto_id is None:
        print("❌ Operação cancelada.")
        return

    # ... (restante do código de atualização com verificações)

def remover_produto():
    print("\n--- Remover Produto (digite '/cancelar' a qualquer momento para cancelar a operação) ---")

    listar_produtos()
    produto_id = input_com_cancelar("\nID do produto a remover: ", int)
    if produto_id is None:
        print("❌ Operação cancelada.")
        return

    confirmacao = input_com_cancelar(f"Tem certeza? (s/n): ")
    if confirmacao is None or confirmacao.lower() != 's':
        print("❌ Operação cancelada.")
        return

    # ... (código de remoção)

def visualizar_logs():
    print("\n--- Logs do Sistema ---")
    cursor.execute('SELECT * FROM logs ORDER BY data_hora DESC')
    logs = cursor.fetchall()

    if not logs:
        print("Nenhum registro de log encontrado.")
    else:
        for log in logs:
            print(f"ID: {log[0]} | Ação: {log[1]} | Produto ID: {log[2]} | "
                  f"Nome: {log[3]} | Quantidade: {log[4]} | Data/Hora: {log[5]}")


def main():
    while True:
        opcao = menu()

        if opcao == '1':
            adicionar_produto()
        elif opcao == '2':
            listar_produtos()
        elif opcao == '3':
            atualizar_produto()
        elif opcao == '4':
            remover_produto()
        elif opcao == '5':
            visualizar_logs()
        elif opcao == '6':
            print("Saindo do sistema...")
            conn.close()
            break
        else:
            print("Opção inválida! Tente novamente.")

if __name__ == "__main__":
    main()


def input_com_cancelar(prompt, tipo=str, default=None):
    """
    Função para input com opção de cancelamento global.
    Retorna:
        - Valor digitado (convertido para o tipo especificado)
        - None se o usuário digitar '/cancelar'
    """
    while True:
        user_input = input(prompt)
        if user_input.strip().lower() == '/cancelar':
            return None
        try:
            if not user_input and default is not None:
                return default
            return tipo(user_input) if user_input else default
        except ValueError:
            print(f"Erro: Digite um valor válido de {tipo.__name__} ou '/cancelar' para cancelar.")




















Banco de dados e tabelas criados com sucesso!

--- Sistema de Estoque Mercadinho ---
1. Adicionar novo produto
2. Listar todos os produtos
3. Atualizar produto
4. Remover produto
5. Visualizar logs
6. Sair
Escolha uma opção: 1

--- Adicionar Novo Produto (digite '/cancelar' a qualquer momento para cancelar a operação) ---
Nome do produto: 1
Categoria: cancelar
Preço: R$ /cancelar
❌ Operação cancelada.

--- Sistema de Estoque Mercadinho ---
1. Adicionar novo produto
2. Listar todos os produtos
3. Atualizar produto
4. Remover produto
5. Visualizar logs
6. Sair
Escolha uma opção: 1

--- Adicionar Novo Produto (digite '/cancelar' a qualquer momento para cancelar a operação) ---
Nome do produto: Feijao
Categoria: Grãos
Preço: R$ 8,99
Erro: Digite um valor válido de float ou '/cancelar' para cancelar.
Preço: R$ 8.99
Quantidade em estoque: 87
✅ Produto 'Feijao' adicionado com sucesso!

--- Sistema de Estoque Mercadinho ---
1. Adicionar novo produto
2. Listar todos os produtos
3. Atualizar pro

KeyboardInterrupt: Interrupted by user